In [44]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from stardist.matching import matching
from PIL import Image
import glob 
import os 
import csv 
from typing import Set

def find_files(directory: str, format: str = '.svs') -> Set[str]:
    """
    Find all folders/subdirectories that contain files with the specified format.
    :param directory: The root directory to search for files.
    :param format: The type of the files to search for (default is '.svs', for the WSI file search).
    :return: A set of directories containing files with the specified format
    """
    svs_directories = set()  # Use a set to store unique parent directories

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(format):
                svs_directories.add(root)  # Use add() to add unique directories

    return svs_directories


In [46]:
# data directory
path_to_predictions = '/home/guoj5/Desktop/200_annotation/temp/cellvit_fold6_latest_723/200_annotation_qa'	#parent directory of '.npy'
path_to_groundtruth = '/home/guoj5/Documents/200_annotation_masks/masks_npy'
# Output LOG File path
# file_path = "/home/guoj5/Desktop/200_annotation/metrics/augment_model_logs/ccellvit_tunefold6_epoch9/cellvit_pred_log.csv"
file_path = '/home/guoj5/Desktop/200_annotation/test1.csv'
os.makedirs(file_path[0:file_path.find(os.path.basename(file_path))], exist_ok=True)

# dataset prediction dir
data_dirs = list(find_files(path_to_predictions, format='.npy'))
print(data_dirs)



for directory in data_dirs:
	print(os.path.basename(directory))
	predictions = glob.glob(os.path.join(directory, '*.npy'))
	
	Log = []
	for prediction in predictions:
		# /path/to/ground truth
		gt = os.path.join(path_to_groundtruth, prediction.split('/')[-1].replace("_contours.npy", ".npy"))
		
        # load predictions and ground truth as I1, I2
		I1 = np.load(prediction).astype(np.uint32)
		I2 = np.load(gt)

        # stats object w/t iou=0.5
		stats = matching(y_true = I2, y_pred = I1, thresh=0.5)
		Log.append([directory, prediction.split('/')[-1], stats.f1, stats.precision, stats.recall, stats.panoptic_quality])

	# Check if the file exists
	if not os.path.exists(file_path):
		# If it doesn't exist, create a new file
		with open(file_path, mode='w', newline='') as file:
			writer = csv.writer(file)
			writer.writerow(['folder', 'predictions', 'f1', 'precision', 'recall', 'pq'])
			for row in Log:
				writer.writerow(row)
		

['/home/guoj5/Desktop/200_annotation/temp/cellvit_fold6_latest_723/200_annotation_qa']
200_annotation_qa


200_annotation_qa


In [33]:
stats

Matching(criterion='iou', thresh=0.5, fp=13, tp=93, fn=32, precision=0.8773584905660378, recall=0.744, accuracy=0.6739130434782609, f1=0.8051948051948052, n_true=125, n_pred=106, mean_true_score=0.6282586669921875, mean_matched_score=0.8444336921938004, panoptic_quality=0.6799336222859172)

In [47]:
df1 = pd.read_csv('/home/guoj5/Desktop/200_annotation/test1.csv')
df2 = pd.read_csv('/home/guoj5/Desktop/200_annotation/test2.csv')

In [48]:
df1.columns

Index(['folder', 'predictions', 'f1', 'precision', 'recall', 'pq'], dtype='object')

In [50]:
df1.mean()

2053335143.py (1): The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


f1           0.789470
precision    0.786122
recall       0.809565
pq           0.658740
dtype: float64

In [51]:
df2.mean()

3587575296.py (1): The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


f1           0.774712
precision    0.815150
recall       0.755956
pq           0.649458
dtype: float64